In [1]:
%pylab inline

from __future__ import print_function
from __future__ import division

import sklearn
import numpy
import scipy
import pandas
import matplotlib
import seaborn

import json
import re

from matplotlib import pyplot
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

Populating the interactive namespace from numpy and matplotlib


In [3]:
def getlinesep(mail):
    lineseps = ['\r\n', '\n']
    return next(x for x in lineseps if 2 * x in mail)

def parse(mail):
    ls = getlinesep(mail)
    bs = 2 * ls
    
    header, body = mail.split(bs, 1)
    header = header.replace(ls + '\t', ' ').replace(ls + ' ', ' ')
    
    header = [(str(x).replace('_', '--'), y) for x, y in [z.split(':', 1) for z in header.split(ls)]]
    body = ('body', body)
    
    return header + [body] + [('my_linesep', ls)]

def try_parse(mail):
    try:
        return parse(mail)
    except ValueError:
        return {'my_unparseable': True, 'my_reason': 'badheader'}
    except StopIteration:
        return {'my_unparseable': True, 'my_reason': 'nosep'}
    


In [117]:
data = pandas.read_csv('data/train.csv', usecols = ['body', 'spam']).fillna('')

# Count Vectorizer

In [149]:
data.index.size

81000

In [215]:
vec = CountVectorizer(token_pattern = r'[a-z]{4,}', min_df = 800)
counts = vec.fit_transform(data.body)

In [216]:
appareances = counts.minimum(1)

In [217]:
Pw = appareances.mean(axis = 0)

In [218]:
Pws = appareances[numpy.where(data.spam)].mean(axis = 0)

In [219]:
Ps = data.spam.mean()

In [220]:
Psw = Pws * Ps / Pw

In [221]:
counts.shape

(81000, 1649)

In [225]:
numpy.sort(Psw)

matrix([[ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [228]:
numpy.sort(numpy.asarray(Psw)[0])

array([ 0.,  0.,  0., ...,  1.,  1.,  1.])

In [210]:
numpy.array(vec.get_feature_names())[[numpy.argsort(numpy.asarray(Psw)[0])[::-1][:10]]]

array([u'dcenter', u'corel', u'dleft', u'rfkindysadvnqw', u'nerasdf',
       u'dreamweaver', u'dbaseline', u'cialis', u'dwindows', u'imageready'], 
      dtype='<U15')

In [77]:
type(appareances)

scipy.sparse.csr.csr_matrix

In [84]:
data['spam'] = data.index > 50

In [91]:
data.spam.sum()

49

In [112]:
numpy.where(data.spam)[0]

array([51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [115]:
appareances[numpy.where(data.spam)]

<49x4108 sparse matrix of type '<type 'numpy.int64'>'
	with 5504 stored elements in Compressed Sparse Row format>

# Common setup

In [2]:
data = pandas.read_csv('data/train.csv', usecols = ['body', 'spam']).fillna('')
vec = CountVectorizer(token_pattern = r'[a-z]{4,}', min_df = 800)
data_norm = data.body.str.replace('<NL>', ' ').replace(r'\\', ' ').str.replace(r'\"', ' ').str.replace(r'"[^"]*"', ' ').str.replace(r'<[^>]*>', ' ')
counts = vec.fit_transform(data_norm)


In [12]:
appareances = counts.minimum(1)

# 100 palabras más spammeras

In [4]:
Pw = appareances.mean(axis = 0)
Pws = appareances[numpy.where(data.spam)].mean(axis = 0)
Ps = data.spam.mean()
Psw = Pws * Ps / Pw

In [11]:
spammeras = numpy.array(vec.get_feature_names())[[numpy.argsort(numpy.asarray(Psw)[0])[::-1][:100]]]

# 100 palabras más hammeras

In [6]:
Pw = appareances.mean(axis = 0)
Pwh = appareances[numpy.where(numpy.invert(data.spam))].mean(axis = 0)
Ph = data.spam.mean()
Phw = Pwh * Ps / Pw



In [10]:
hammeras = numpy.array(vec.get_feature_names())[[numpy.argsort(numpy.asarray(Phw)[0])[::-1][:100]]]

# Tabla final

In [8]:
spammeras_hammeras = numpy.array([[spammeras, hammeras]]).flatten()
vec_final = CountVectorizer(min_df = 1)
vec_final.fit_transform(spammeras_hammeras.tolist())
counts_total = vec_final.transform(data_norm)
appareances_total = counts_total.minimum(1)


# Pruebas

In [ ]:
array = numpy.asarray([5,4,3,2,1])
indexes = numpy.argsort(array)
array[indexes]

In [273]:
data.head().body.str.replace(r'<[^>]*>', ' ')

0    chris,  thanks for the invitation. the evening...
1     \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \...
2    we have determined (vickers and tycholiz and h...
3    yes,  it is a great accomplishment.  please do...
4    fyi  ---------------------- forwarded by richa...
Name: body, dtype: object

In [ ]:
map(print, data_norm[data_norm.str.contains(' weight ')].head())

In [ ]:
appareances.todense()

In [ ]:
vec.get_feature_names()[numpy.argmax(Psw)]

In [ ]:
type(appareances)